# Importing some packages

In [74]:
import pandas as pd
import numpy as np
import re
import string 

# Loading the data

In [30]:
df = pd.read_csv('data/estate.csv')
df.head()

,description,address,usd_price,amd_price,rub_price,seller_type,attributes,total_description,places_nearby,posted_date,renewed_date,seller_id,estate_type,estate_id
0,"2 room apartment, Pushkin Street, 48 sq.m., 3/...","Pushkin Street, Yerevan",165000,64779000,15386936,00591Agency,"[('Construction Type', 'Panels'), ('New Constr...",A 2-room apartment with an area of 48 square m...,Opera Theatre; Republic Square; Cascade; Verni...,2024-02-03,2024-04-19 00:00:00,capitalcity,apartment,/en/item/20602332
1,"3 room apartment on Yervand Kochar street, 90 ...","Yervand Kochar street, Yerevan",170000,66742000,15853207,307992Agency,"[('Construction Type', 'Panels'), ('New Constr...",For sale: 3-room E. On Kochar street. The apar...,Saint Gregory the Illuminator Cathedral; Verni...,2023-02-13,2024-04-18 00:00:00,658934,apartment,/en/item/19008391
2,"2 room apartment on Shahumyan 1st street, 62 s...","Shahumyan 1st street 4, Yerevan",88000,34548800,8206366,NaN,"[('Construction Type', 'Stone'), ('New Constru...",An apartment in Araratra 1 block is for sale w...,Unknown,2023-09-27,2024-04-20 00:00:00,21606,apartment,/en/item/20049462
3,"3 room apartment on Moldovakan street, 68 sq.m...","Մոլդովական փողոց 48/3, Yerevan",130000,51038000,12123040,NaN,"[('Construction Type', 'Panels'), ('New Constr...","In the 2nd block of Nor Nork, a 3-room apartme...",Unknown,2023-09-27,2024-04-20 00:00:00,1375140,apartment,/en/item/20050503
4,"3 room apartment, Vardanants Street, 68 sq.m.,...","Vardanants Street, Yerevan",165000,64779000,15386936,3k48Agency,"[('Construction Type', 'Panels'), ('New Constr...",(code 3k48) 3-room apartment for sale in the c...,Saint Gregory the Illuminator Cathedral; Verni...,2023-10-20,2024-04-18 00:00:00,2572754,apartment,/en/item/20153128


# EDA

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7491 entries, 0 to 7490
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        7491 non-null   object
 1   address            7491 non-null   object
 2   usd_price          7491 non-null   object
 3   amd_price          7491 non-null   object
 4   rub_price          7491 non-null   object
 5   seller_type        6472 non-null   object
 6   attributes         7491 non-null   object
 7   total_description  7491 non-null   object
 8   places_nearby      7491 non-null   object
 9   posted_date        7491 non-null   object
 10  renewed_date       7491 non-null   object
 11  seller_id          7491 non-null   object
 12  estate_type        7491 non-null   object
 13  estate_id          7491 non-null   object
dtypes: object(14)
memory usage: 819.5+ KB


### Removing rows for which the price is not available

In [32]:
df = df[df['usd_price']!='Unknown'].reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7085 entries, 0 to 7084
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   description        7085 non-null   object
 1   address            7085 non-null   object
 2   usd_price          7085 non-null   object
 3   amd_price          7085 non-null   object
 4   rub_price          7085 non-null   object
 5   seller_type        6066 non-null   object
 6   attributes         7085 non-null   object
 7   total_description  7085 non-null   object
 8   places_nearby      7085 non-null   object
 9   posted_date        7085 non-null   object
 10  renewed_date       7085 non-null   object
 11  seller_id          7085 non-null   object
 12  estate_type        7085 non-null   object
 13  estate_id          7085 non-null   object
dtypes: object(14)
memory usage: 775.0+ KB


In [64]:
df[df['renewed_date']=='Unknown']

,description,address,seller_type,total_description,places_nearby,posted_date,renewed_date,seller_id,estate_type,estate_id,...,dishwasher,yard view,view of Ararat,park view,city view,street view,concierge,playground,intercom entry,sqm_price_usd
9,"3 room apartment, Ադոնցի փողոց, 101 sq.m., 3/8...","Ադոնցի փողոց 4/3, Երևան",individual,It is possible to sell with or without furnitu...,Unknown,2024-04-19,Unknown,165632,apartment,/en/item/20922312,...,1,1,0,0,0,1,0,1,0,2772.277228
34,"2 room apartment on Zakaria Sarkavag street, 6...","Zakaria Sarkavag street, Yerevan",agency,A 2-room apartment is for sale in one of the s...,Unknown,2024-04-19,Unknown,2470975,apartment,/en/item/20919760,...,0,1,0,0,0,0,0,0,0,1159.420290
62,"4 room apartment in a new building, Komitas Av...","Проспект Комитаса 59, Yerevan",agency,Многофункциональный жилой комплекс состоит из ...,Russian-Armenian University; Barekamutyun Metr...,2024-04-20,Unknown,2999545,apartment,/en/item/20923773,...,0,1,1,0,1,0,1,1,1,2167.785714
95,"3 room apartment, Gusan Sheram Street, 80 sq.m...","Gusan Sheram Street, Yerevan",agency,A converted 3/4 room apartment is for sale in ...,Unknown,2024-04-19,Unknown,1294205,apartment,/en/item/20921675,...,0,0,0,0,0,0,0,0,0,1275.000000
99,"1 room apartment on Lepsius street, 31 sq.m., ...","Lepsius street, Yerevan",agency,"1-room apartment for sale, 14 Krug Street, the...",Unknown,2024-04-19,Unknown,47627,apartment,/en/item/20922289,...,0,1,0,0,0,0,0,1,0,1580.645161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6948,3 room apartment in a new building on Verin An...,"Verin Antarayin street, Yerevan",agency,A 3-room apartment is for sale on Verin Antara...,Marshal Baghramyan Metro Station; Lovers Park;...,2024-04-19,Unknown,2933967,apartment,/en/item/20922250,...,1,1,1,0,0,1,0,1,1,3125.000000
6994,"3 room apartment, Sayat Nova avenue, 88 sq.m.,...","Sayat Nova avenu 37, Երևան",individual,"An apartment is for sale in Sayat Nova Avenue,...",Opera Theatre; Yeritasardakan Metro Station; R...,2024-04-19,Unknown,48218,apartment,/en/item/20922297,...,1,1,0,0,0,1,0,0,0,2443.181818
7028,3 room apartment in a new building on Leningra...,"Լենինգրադյան փողոց, Yerevan",agency,A 3-room apartment in a newly built building i...,Kievyan Bridge,2024-04-17,Unknown,charents,apartment,/en/item/20914116,...,0,1,1,1,1,1,1,1,1,1776.315789
7048,"2 room apartment, Shopron street 1st lane, 61 ...","Շոպրոնի 1-ին նրբանցք, Երևան",agency,An apartment is for sale in the 6th block of N...,Unknown,2024-04-18,Unknown,2997367,apartment,/en/item/20917378,...,0,0,0,0,0,0,0,0,0,1442.622951


### Setting the dates to right format

In [65]:
def create_dates(x):
    try:
        return pd.to_datetime(x)
    except:
        return 'not renewed'

df['posted_date'] = pd.to_datetime(df['posted_date'])
df['renewed_date'] = df['renewed_date'].apply(create_dates)

### Splitting attributes column to features

In [33]:
possible_attributes_list = list(
    set(
        [
            j.replace('[(', '').replace(')]', '').replace('(', '').\
            replace("'", '').split(',')[0].strip()
            for i in range(df.shape[0]) 
            for j in df.iloc[i]['attributes'].split('),')
        ]
    )
)

def split_attributes(x):
    attrs_tuple_list = x.split('),')
    attrs_tuple_list = [
        i.replace('[(', '').replace(')]', '').replace('(', '').\
        replace("'", '').split(',') 
        for i in attrs_tuple_list
    ]
    attrs_dict = {i[0].strip(): ', '.join([j.strip() for j in i[1:]]) for i in attrs_tuple_list}
    return attrs_dict


for possible_attribute in possible_attributes_list:
    df[possible_attribute] = df['attributes'].apply(
        lambda x: split_attributes(x)[possible_attribute] 
        if possible_attribute in split_attributes(x) 
        else 'Unknown'
    )
    print(f"{possible_attribute} values: {list(df[possible_attribute].unique())}")

df.drop(columns='attributes', inplace=True)


Floors in the Building values: ['9', '16', '5', '10', '4', '3', '8', '11', '14', '21', '15', '6', '23', '1', '7', '18', '12', '20', '13', '17', '2', '32+', '19', '22', '30']
Floor Area values: ['48 sq.m.', '90 sq.m.', '62 sq.m.', '68 sq.m.', '88 sq.m.', '86 sq.m.', '50 sq.m.', '79 sq.m.', '101 sq.m.', '66 sq.m.', '46 sq.m.', '92 sq.m.', '98 sq.m.', '115 sq.m.', '65 sq.m.', '75 sq.m.', '47 sq.m.', '72 sq.m.', '80 sq.m.', '37 sq.m.', '94 sq.m.', '89 sq.m.', '93 sq.m.', '104 sq.m.', '100 sq.m.', '30 sq.m.', '73 sq.m.', '165 sq.m.', '82 sq.m.', '69 sq.m.', '172 sq.m.', '175 sq.m.', '77 sq.m.', '132 sq.m.', '54 sq.m.', '95 sq.m.', '57 sq.m.', '67 sq.m.', '117 sq.m.', '150 sq.m.', '51 sq.m.', '70 sq.m.', '105 sq.m.', '128 sq.m.', '107 sq.m.', '85 sq.m.', '140 sq.m.', '45 sq.m.', '58 sq.m.', '120 sq.m.', '118 sq.m.', '71 sq.m.', '49 sq.m.', '44 sq.m.', '122 sq.m.', '56 sq.m.', '230 sq.m.', '60 sq.m.', '74 sq.m.', '108 sq.m.', '64 sq.m.', '78 sq.m.', '96 sq.m.', '42 sq.m.', '63 sq.m.', '31 sq.

#### Parking splitting

In [34]:
possible_parking_list = list(set(', '.join([i.lower().strip() for i in df['Parking'].unique()]).split(',')))
possible_parking_list = list(set([i.strip() for i in possible_parking_list if i.strip()!='unknown']))

def get_window_view_list(x):
    if len(x.split(',')) > 1:
        return [i.lower().strip() for i in x.split(',')]
    else:
        return [x.lower().strip()]

for possible_parking in possible_parking_list:
    df[possible_parking] = df['Parking'].apply(lambda x: 1 if possible_parking in get_window_view_list(x) else 0)
    print(f"{possible_parking} values: {list(df[possible_parking].unique())}")

df.drop(columns='Parking', inplace=True)

garage values: [0, 1]
covered parking values: [0, 1]
outdoor parking values: [1, 0]


#### Appliances splitting

In [35]:
possible_appliances_list = list(set(', '.join([i.lower().strip() for i in df['Appliances'].unique()]).split(',')))
possible_appliances_list = list(set([i.strip() for i in possible_appliances_list if i.strip()!='unknown']))

def get_window_view_list(x):
    if len(x.split(',')) > 1:
        return [i.lower().strip() for i in x.split(',')]
    else:
        return [x.lower().strip()]

for possible_appliance in possible_appliances_list:
    df[possible_appliance] = df['Appliances'].apply(lambda x: 1 if possible_appliance in get_window_view_list(x) else 0)
    print(f"{possible_appliance} values: {list(df[possible_appliance].unique())}")

df.drop(columns='Appliances', inplace=True)

stove values: [1, 0]
air conditioner values: [1, 0]
drying machine values: [0, 1]
fridge values: [1, 0]
washing machine values: [1, 0]
dishwasher values: [0, 1]


#### creating a new category for Elevator feature

In [36]:
df['Elevator'] = df['Elevator'].apply(lambda x: 'absent' if x.lower() in ['unknown', 'not available'] else x.lower())

#### creating a new category for Furniture feature

In [37]:
df['Furniture'] = df['Furniture'].apply(lambda x: 'absent' if x.lower() in ['unknown', 'not available', 'by agreement'] else x.lower())

#### Window Views splitting

In [38]:
possible_window_view_list = list(set(', '.join([i.lower().strip() for i in df['Window Views'].unique()]).split(',')))
possible_window_view_list = list(set([i.strip().replace('ararat', 'Ararat') for i in possible_window_view_list if i.strip()!='unknown']))

def get_window_view_list(x):
    if len(x.split(',')) > 1:
        return [i.lower().strip().replace('ararat', 'Ararat') for i in x.split(',')]
    else:
        return [x.lower().strip().replace('ararat', 'Ararat')]

for possible_window_view in possible_window_view_list:
    df[possible_window_view] = df['Window Views'].apply(lambda x: 1 if possible_window_view in get_window_view_list(x) else 0)
    print(f"{possible_window_view} values: {list(df[possible_window_view].unique())}")

df.drop(columns='Window Views', inplace=True)

yard view values: [1, 0]
view of Ararat values: [0, 1]
park view values: [0, 1]
city view values: [0, 1]
street view values: [0, 1]


#### creating new category for Renovation feature

In [39]:
df['Renovation'] = df['Renovation'].apply(lambda x: 'absent' if x.lower() in ['unknown', 'no renovation'] else x.lower())

#### Floor Area extraction

In [40]:
def get_sqm(x):
    try:
        return int(re.findall(r'\d?\d?\d?\.?\d?\d?', x)[0])
    except:
        return 999
    
df['Floor Area'] = df['Floor Area'].apply(get_sqm)

#### Floors in the Building extraction

In [41]:
def get_building_floors_num(x):
    try:
        return int(re.findall(r'\d?\d?', x)[0])
    except:
        return 999
    
df['Floors in the Building'] = df['Floors in the Building'].apply(get_building_floors_num)

#### creating a new category in Balcony feature

In [42]:
df['Balcony'] = df['Balcony'].apply(lambda x: 'absent' if x.lower() in ['unknown', 'not available'] else x.lower())

#### Ceiling height extraction

In [43]:
def get_ceiling_height(x):
    try:
        return float(re.findall(r'\d.?\d?\d?', x)[0])
    except:
        return 999
    
df['Ceiling Height'] = df['Ceiling Height'].apply(get_ceiling_height)

#### Floor extraction

In [44]:
def get_floor_num(x):
    try:
        return int(re.findall(r'\d?\d?', x)[0])
    except:
        return 999
    
df['Floor'] = df['Floor'].apply(get_floor_num)

#### "The House Has" column splitting

In [45]:
possible_has_list = list(set(', '.join([i.lower() for i in df['The House Has'].unique()]).split(',')))
possible_has_list = list(set([i.strip() for i in possible_has_list if i.strip()!='unknown']))

def get_has_list(x):
    if len(x.split(',')) > 1:
        return [i.lower().strip() for i in x.split(',')]
    else:
        return [x.lower().strip()]

for possible_has in possible_has_list:
    df[possible_has] = df['The House Has'].apply(lambda x: 1 if possible_has in get_has_list(x) else 0)
    print(f"{possible_has} values: {list(df[possible_has].unique())}")

df.drop(columns='The House Has', inplace=True)

concierge values: [0, 1]
playground values: [0, 1]
intercom entry values: [0, 1]


### Getting square meter prices

In [51]:
# removing rows where floor area is missing
df = df[df['Floor Area']!=999].reset_index(drop=True)

df['usd_price'] = df['usd_price'].apply(float)

df['sqm_price_usd'] =  df['usd_price'] / df['Floor Area']
# df['sqm_price_amd'] = df['amd_price'] / df['Floor Area']
# df['sqm_price_rub'] = df['rub_price'] / df['Floor Area']

df.drop(columns=['usd_price', 'amd_price', 'rub_price'], inplace=True)

### "Tuning" seller_type feature

In [61]:
df['seller_type'].fillna('individual', inplace=True)
df['seller_type'] = df['seller_type'].apply(lambda x: 'agency' if 'Agency' in x else 'individual')

### Dealing with the estate_id

In [68]:
df['estate_id'] = df['estate_id'].apply(lambda x: int(x.replace('/en/item/', '')))

### Showing possible places_nearby and removing the feature

In [98]:
places_by_list = list(
    set(
        [
            j.strip() for i in range(df.shape[0]) 
            for j in df.iloc[i]['places_nearby'].split(';') 
            if j!='Unknown'
        ]
    )
)
places_by_list

['Garegin Nzhdeh Square',
 'Marshal Baghramyan Metro Station',
 'General Andranik Metro Station',
 'Yerevan Railway Station',
 'Republic Square',
 'Orange Fitness Premium Club',
 'Matenadaran',
 'Barekamutyun Metro Station',
 'Kievyan Bridge',
 'Reebok Sports Club',
 'Dalma Garden Mall',
 'Russian-Armenian University',
 'Cascade',
 'Gortsaranayin Metro Station',
 'Saint Gregory the Illuminator Cathedral',
 'Yerevan State Medical University',
 'Yeritasardakan Metro Station',
 'Lovers Park',
 'Vernissage',
 'Yerevan Mall',
 'Vartanants Square',
 'Victory Park',
 'Shengavit Metro Station',
 'Opera Theatre']

In [100]:
df.drop(columns='places_nearby', inplace=True)

### Dealing with address feature

In [101]:
# df[df['address'].apply(lambda x: True if 'Yerevan Highway' in x else False)]

In [105]:
region_map_dict = {
    'Երևան': 'Yerevan', 'Հրազդան': 'Hrazdan', 'Վանաձոր': 'Vanadzor',
    'Ереван': 'Yerevan', 'Աբովյան': 'Abovyan', 'Առինջ': 'Arinj',
    'Արտաշատ': 'Artashat', 'Ծաղկաձոր': 'Tsaghkadzor', 'Արարատ': 'Ararat',
    'Չարենցավան': 'Charentsavan', 'Մասիս': 'Masis', 'Քասախ': 'Qasakh',
    'Lorri': 'Lori', 'Гюмри': 'Gyumri', 'Սևան': 'Sevan', 'Ванадзор': 'Vanadzor',
    'Ախուրյան': 'Akhuryan', 'Էջմիածին քաղաք': 'Vagharshapat', 'Մարտունի': 'Martuni', 
    'Նոր Գեղի': 'Nor Geghi', 'Մերձավան': 'Merdzavan', 'Раздан': 'Hrazdan',
    'Ավան': 'Avan', 'Мецамор': 'Metsamor', 'Դիլիջան': 'Dilijan', 'Ջերմուկ': 'Jermuk',
    'Եղվարդ': 'Yeghvard', 'Նոր Հաճըն': 'Nor Hachin', 'Абовян': 'Abovyan', 
    'Արմավիր': 'Armavir', 'Բյուրեղավան': 'Byureghavan', 'Зовуни': 'Zovuni', 
    'Գյումրի': 'Gyumri', 'Вагаршапат': 'Vagharshapat', 'Դաշտավան': 'Dashtavan', 
    'Էջմիածին': 'Vagharshapat', 'Ташир': 'Tashir', 'Ехвард': 'Yeghvard', 'Փշատավան': 'Pshatavan', 
    'Աշտարակ': 'Ashtarak', 'Ejmiatsin': 'Vagharshapat', 'Վեդի': 'Vedi', 
    'Վարդենիս': 'Vardenis', 'Փարաքար': 'Parakar', 'Սիսիան': 'Sisian', 'Գավառ': 'Gavar',
    'Շահումյան': 'Shahumyan', 'Վաղարշապատ': 'Vagharshapat', 'Քաջարան': 'Qajaran',
    'Իջևան': 'Ijevan', 'Ջրվեժ': 'Jrvej', 'Կապան': 'Kapan', 'Ապարան': 'Aparan', 
    'Արթիկ': 'Artik', 'Թաիրով': 'Tairov', 'Арташат': 'Artashat', 'Մոնթեավան': 'Monteavan', 
    'Գորիս': 'Goris', 'Ավշար': 'Avshar', 'Զովունի': 'Zovuni', 'Масис': 'Masis'
}

In [104]:
def split_address(x):
    if '›' in x:
        split_result = [
            region_map_dict[i.strip()] if i.strip() in region_map_dict 
            else i.strip() for i in x.split('›')
        ]
        return split_result
    else:
        comma_split = [
            region_map_dict[i.strip()] if i.strip() in region_map_dict
            else i.strip() for i in x.split(',')
        ]
        if len(comma_split) == 1:
            return x
        else:
            if len(comma_split[0]) > len(comma_split[1]):
                return comma_split[::-1]
            else:
                if 'street' in comma_split[0].lower() or 'փողոց' in comma_split[0]:
                    return comma_split[::-1]
                else:
                    return comma_split

df['address'].apply(lambda x: split_address(x)[0]).unique()

array(['Yerevan', 'Shirak', 'Hrazdan', 'Zovuni', 'Vanadzor', 'Kotayk',
       'Abovyan', 'Arinj', 'Gyumri', 'Artashat', 'Tsaghkadzor',
       'ՔՇՀ-4 22', 'Ararat', 'U', '3-րդ թաղամաս', 'Masis', 'Vagharshapat',
       'Armavir', 'Qasakh', 'Lori', 'Sevan', 'Aragatsotn', 'Akhuryan',
       'Yerevan Highway', 'Tavush', 'Martuni', 'Nor Geghi', 'Merdzavan',
       'Artaxata (Artashat)', 'Avan', 'Metsamor', 'Dilijan', 'Jermuk',
       'Yeghvard', 'Nor Hachin', 'Spitak', 'Charentsavan', 'Byureghavan',
       'Dashtavan', 'Tashir', 'Garni', 'Ashtarak', 'Pshatavan', 'Syunik',
       'Sevan town', 'Vedi', 'Mrgashen village', 'Vayots Dzor',
       'Vardenis', 'Parakar', 'Sisian', 'Gavar', 'Shahumyan', 'Qajaran',
       'Vayk', 'Ijevan', '26 St', 'Jrvej', 'Նոր Նորքի 9-րդ զանգված',
       'Kapan', 'Aparan', 'Artik', 'Gegharkunik', 'Tairov', 'Monteavan',
       'Goris', 'Avshar', 'Argavand', '8-й район'], dtype=object)

Checking for distribution among regions/cities

In [106]:
df['address'].apply(lambda x: split_address(x)[0]).value_counts()

address
Yerevan            6670
Vanadzor             46
Kotayk               41
Gyumri               35
Abovyan              31
                   ... 
Akhuryan              1
Spitak                1
Yerevan Highway       1
Martuni               1
8-й район             1
Name: count, Length: 69, dtype: int64

Checking for street names

In [95]:
df['address'].apply(lambda x: split_address(x)[1]).unique()

array(['Pushkin Street', 'Yervand Kochar street',
       'Shahumyan 1st street 4', ..., 'Квартал Кентрон 47',
       'Улица Нвера Сафаряна', 'Հայկ Գյուլիքևխյանի փողոց'], dtype=object)

Creating Yerevan districts' street names

In [138]:
ajapnyak_text = '''Հալաբյան փողոց
Գրիգոր Հասրաթյան փողոց
Շիրազի փողոց
Աշտարակի խճուղի
Չաուշի փողոց
Սիլիկյան նոր խճուղի
Սեբաստիա փողոց
Լենինգրադյան փողոց
Բեկնազարյան փողոց
Բաշինջաղյան փողոց
Մազմանյան փողոց
Շինարարների փողոց
Մարգարյան փողոց
Աբելյան փողոց'''

avan_text = '''Բաբաջանյան փողոց
Աճառյան փողոց
Ռուբինյանց փողոց
Խուդյակովի փողոց
Բագրևանդի փողոց'''

arabkir_text = '''Բաղրամյան պողոտա
Կասյան փողոց
Քոչարի փողոց
Վաղարշյան փողոց
Սարալանջի փողոց
Կոմիտասի պողոտա
Կիևյան փողոց
Ազատության պողոտա
Թբիլիսյան խճուղի
Օրբելի եղբայրների փողոց
Ավետիսյան փողոց
Համբարձումյան փողոց
Հակոբյան փողոց
Խաչատրյան փողոց
Գյուլբենկյան փողոց
Փափազյան փողոց
Մամիկոնյանց փողոց
Տիգրանյան փողոց
Շահսուվարյան փողոց
Վրացական փողոց
Բաբայան փողոց
Գրիբոյեդովի փողոց
Շիրվանզադեի փողոց
Զարյան փողոց
Ադոնցի փողոց'''

davtashen_text = '''Միկոյան փողոց 
Սասնա Ծռերի փողոց
Պետրոսյան փողոց
Փիրումյան փողոց
Աղաբաբյան փողոց
Գևորգյան փողոց'''

erebuni_text = '''
Տիգրան Մեծի պողոտա
Արցախի պողոտա
Այվազովսկու փողոց
Էրեբունու փողոց
Դավիթ Բեկի փողոց
Ռոստովյան փողոց
Գարեգին Նժդեհի փողոց
Խորենացու փողոց
Բելինսկու փողոց
Ազատամարտիկների պողոտա
Արին-Բերդի փողոց
Դաշտենցի փողոց
Նուբարաշենի խճուղի
Սասունցի Դավթի փողոց
Խաղաղ Դոնի փողոց
Տիտոգրադյան փողոց
Աթոյան փողոց'''

kentron = '''Բաղրամյան պողոտա
Մաշտոցի պողոտա
Կորյունի փողոց
Սարյան փողոց
Ամիրյան փողոց
Տիգրան Մեծի պողոտա
Խանջյան փողոց
Աբովյան փողոց
Նալբանդյան փողոց
Մոսկովյան փողոց
Հերացու փողոց
Մյասնիկյան պողոտա
Խորենացու փողոց
Պռոշյան փողոց
Ագաթանգեղոսի փողոց
Արշակունյաց պողոտա
Իսակովի պողոտա
Չարենցի փողոց
Նար-Դոսի փողոց
Սայաթ-Նովայի պողոտա
Ծիծեռնակաբերդի խճուղի
Գետառի փողոց
Պարոնյան փողոց
Սարալանջի փողոց
Աթենքի փողոց
Գրիգոր Լուսավորչի փողոց
Վարդանանց փողոց
Իսրայելյան փողոց
Ձորափի փողոց
Արամի փողոց
Բուզանդի փողոց
Պուշկինի փողոց
Ֆիրդուսի փողոց
Զորյանի փողոց
Կողբացու փողոց
Տպագրիչների փողոց
Անտառային փողոց
Քաջազնունու փողոց
Տերյան փողոց
Հանրապետության փողոց
Մկրտչյան փողոց 
Մելիք-Ադամյան փողոց
Զաքյան փողոց
Լեոյի փողոց
Քոչարի փողոց
Մանուկյան փողոց
Դեմիրճյան փողոց
Արգիշտիի փողոց
Շարիմանյան փողոց
Բեյրութի փողոց
Իտալիայի փողոց
Սարգսյան փողոց'''

malatia_sebastia_text = '''Սեբաստիայի փողոց
Ծովակալ Իսակովի պողոտա
Բաբաջանյան փողոց
Րաֆֆու փողոց
Շերամի փողոց
Ամերիկյան փողոց
Տիչինայի փողոց
Օհանովի փողոց
Զորավար Անդրանիկի փողոց
Լենինգրադյան փողոց
Շիրակի փողոց 
Մալաթիայի փողոց 
Արարատյան փողոց'''

nor_norq_text = '''Գայի պողոտա
Մյասնիկյանի պողոտա
Թևոսյան փողոց
Բագրևանդի փողոց
Բ.Մուրադյան փողոց
Քոչինյան փողոց
Դավիթ Բեկի փողոց
Միկոյան փողոց
Գյուլիքևխյան փողոց 
Գյուրջյան փողոց
Գալշոյան փողոց
Շոպրոնի փողոց
Սաֆարյան փողոց'''

norq_marash_text = '''Արմենակյան փողոց
Հովսեփյան փողոց'''

nubarashen_text = '''Նուբարաշենի խճուղի'''

shengavit_text = '''Շիրակի փողոց
Արտաշեսյան պողոտա
Արտաշատի խճուղի
Արշակունյաց պողոտա
Գագեգին Նժդեհի փողոց
Բագրատունյաց փողոց
Թամանցիների փողոց
Եղբայրության փողոց 
Մանանդյան փողոց
Մանթաշյան փողոց
Մայիսի 9-ի փողոց
Ֆրունզեի փողոց 
Չեխովի փողոց
Արարատյան փողոց 
Նորագավիթի 1-ին փողոց'''

qanaqer_zeytun_text = '''Դավիթ Անհաղթի փողոց
Ռուբինյանց փողոց
Սևակի փողոց
Զաքարիա Քանաքեռցու փողոց
Հասրաթյան փողոց
Ուլնեցու փողոց
Քանաքեռ 1-ին փողոց'''#/Սարկավագի/

Creating Russian to English letters converter dictionary

In [150]:
temp_text = '''⟨б⟩, ⟨в⟩, ⟨г⟩, ⟨д⟩, ⟨ж⟩, ⟨з⟩, ⟨к⟩, ⟨л⟩, ⟨м⟩, ⟨н⟩, 
⟨п⟩, ⟨р⟩, ⟨с⟩, ⟨т⟩, ⟨ф⟩, ⟨х⟩, ⟨ц⟩, ⟨ч⟩, ⟨ш⟩, ⟨щ⟩, ⟨а⟩, ⟨е⟩, ⟨ё⟩, 
⟨и⟩, ⟨о⟩, ⟨у⟩, ⟨ы⟩, ⟨э⟩, ⟨ю⟩, ⟨я⟩, ⟨й⟩, ⟨ъ⟩, ⟨ь⟩'''

temp_text = temp_text.replace('⟨', '').replace('⟩', '').replace('\n', '').split(',')
temp_text = [i.strip() for i in temp_text]

rus2eng_letters_list = [
    'b', 'v', 'g', 'd', 'jհ', 'z', 'k', 'l', 'm', 'n', 'p', 
    'r', 's', 't', 'f', 'kh', 'ts', 'ch', 'sh', 'sh', 'a', 'e', 
    'e', 'i', 'o', 'u', 'i', 'e', 'yu', 'ya', 'i', '', ''
]

rus2eng_letters_dict = {temp_text[i]: rus2eng_letters_list[i] for i in range(len(temp_text))}
# rus2eng_letters_dict.update({' ':' '})
print(rus2eng_letters_dict)

{'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'ж': 'jհ', 'з': 'z', 'к': 'k', 'л': 'l', 'м': 'm', 'н': 'n', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'sh', 'а': 'a', 'е': 'e', 'ё': 'e', 'и': 'i', 'о': 'o', 'у': 'u', 'ы': 'i', 'э': 'e', 'ю': 'yu', 'я': 'ya', 'й': 'i', 'ъ': '', 'ь': ''}


Creating Armenian to English letter conversion dictionary

In [161]:
temp_text = '''ա a,	ճ ch, բ	b, մ m, գ g, յ y,
դ d, ն n, ե	e, շ sh, զ z, ո	o, է e,	չ ch,
ը e, պ p, թ t, ջ j, ժ jh, ռ	r, ի i, ս s,
լ l, վ v, խ	kh, տ t, ծ ts, ր r, կ k, ց ts,
հ h, ու u, ձ dz, փ p, ղ gh, ք k, և ev, օ o,	
ֆ f'''

temp_text = temp_text.replace('\n', '').replace('\t', ' ').split(',')
temp_text = [i.strip().split(' ') for i in temp_text]
len(temp_text)

arm2eng_letters_dict = {elem[0]: elem[1] for elem in temp_text}
# arm2eng_letters_dict.update({' ':' '})
print(arm2eng_letters_dict)

{'ա': 'a', 'ճ': 'ch', 'բ': 'b', 'մ': 'm', 'գ': 'g', 'յ': 'y', 'դ': 'd', 'ն': 'n', 'ե': 'e', 'շ': 'sh', 'զ': 'z', 'ո': 'o', 'է': 'e', 'չ': 'ch', 'ը': 'e', 'պ': 'p', 'թ': 't', 'ջ': 'j', 'ժ': 'jh', 'ռ': 'r', 'ի': 'i', 'ս': 's', 'լ': 'l', 'վ': 'v', 'խ': 'kh', 'տ': 't', 'ծ': 'ts', 'ր': 'r', 'կ': 'k', 'ց': 'ts', 'հ': 'h', 'ու': 'u', 'ձ': 'dz', 'փ': 'p', 'ղ': 'gh', 'ք': 'k', 'և': 'ev', 'օ': 'o', 'ֆ': 'f'}


Creating street-district dictionary

In [162]:
district_text_list = [
    ajapnyak_text, avan_text, arabkir_text, davtashen_text, erebuni_text, kentron, 
    malatia_sebastia_text, nor_norq_text, norq_marash_text, nubarashen_text, 
    shengavit_text, qanaqer_zeytun_text
]

district_names = [
    'ajapnyak', 'avan', 'arabkir', 'davtashen', 'erebuni', 'kentron', 
    'malatia_sebastia', 'nor_norq', 'norq_marash', 'nubarashen', 
    'shengavit', 'qanaqer_zeytun'
]

def convert_arm2eng_street(street):
    street = street.lower()
    street = list(street)
    for idx in range(len(street)):
        if idx == 0 and street[idx] == 'ո' and street[idx+1] != 'ւ':
            street[idx] = 'vo'
        elif street[idx] in [' ', '-', '.', '1', 's', 't', '9']:
            pass
        elif street[idx] == 'ւ':
            street[idx] = ''
        elif idx < len(street) - 1:
            if street[idx] == 'ո' and street[idx+1] == 'ւ':
                street[idx] = arm2eng_letters_dict['ու']
            else:
                street[idx] = arm2eng_letters_dict[street[idx]]
        else:
            street[idx] = arm2eng_letters_dict[street[idx]]
    return ''.join(street)

street_district_dict = {}

for idx in range(len(district_text_list)):
    district_streets = district_text_list[idx].replace('\n', ',').split(',')
    district_streets = [
        i.replace('փողոց', '').replace('պողոտա', '').replace('խճուղի', '').\
        replace('1-ին', '1st') .replace('9-ի', '9').strip()
        for i in district_streets
    ]
            
    eng_streets = [convert_arm2eng_street(district_street) for district_street in district_streets]
    temp_street_district_dict = {street: district_names[idx] for street in eng_streets}
    street_district_dict.update(temp_street_district_dict)

street_district_dict

{'halabyan': 'ajapnyak',
 'grigor hasratyan': 'ajapnyak',
 'shirazi': 'ajapnyak',
 'ashtaraki': 'ajapnyak',
 'chaushi': 'ajapnyak',
 'silikyan nor': 'ajapnyak',
 'sebastia': 'ajapnyak',
 'leningradyan': 'malatia_sebastia',
 'beknazaryan': 'ajapnyak',
 'bashinjaghyan': 'ajapnyak',
 'mazmanyan': 'ajapnyak',
 'shinararneri': 'ajapnyak',
 'margaryan': 'ajapnyak',
 'abelyan': 'ajapnyak',
 'babajanyan': 'malatia_sebastia',
 'acharyan': 'avan',
 'rubinyants': 'qanaqer_zeytun',
 'khudyakovi': 'avan',
 'bagrevandi': 'nor_norq',
 'baghramyan': 'kentron',
 'kasyan': 'arabkir',
 'kochari': 'kentron',
 'vagharshyan': 'arabkir',
 'saralanji': 'kentron',
 'komitasi': 'arabkir',
 'kievyan': 'arabkir',
 'azatutyan': 'arabkir',
 'tbilisyan': 'arabkir',
 'orbeli eghbayrneri': 'arabkir',
 'avetisyan': 'arabkir',
 'hambardzumyan': 'arabkir',
 'hakobyan': 'arabkir',
 'khachatryan': 'arabkir',
 'gyulbenkyan': 'arabkir',
 'papazyan': 'arabkir',
 'mamikonyants': 'arabkir',
 'tigranyan': 'arabkir',
 'shahsuvary

Keeping only Yerevan apartments

In [110]:
def apply_Yerevan(x):
    if 'Երևան' in x or 'Ереван' in x:
        return x.replace('Երևան', 'Yerevan').replace('Ереван', 'Yerevan')
    else:
        return x

df['address'] = df['address'].apply(apply_Yerevan)
df = df[df['address'].apply(lambda x: True if 'Yerevan' in x else False)].reset_index(drop=True)


In [170]:
df['address'].apply(get_street)[df['address'].apply(lambda x: True if 'улица' in x.lower() else False)]

59                    нор ареша,
175                 мамиконянца,
376             карапета улнеци,
397         маршала бабаджаняна,
507            езник кохбаци 42,
                  ...           
6245         андраника зоравара,
6331         григора асратяна 6,
6358              грибоедова 58,
6572    захария саркавага 127/1,
6670             нвера сафаряна,
Name: address, Length: 78, dtype: object

In [200]:
'Шоссе'.lower()

'шоссе'

In [205]:
def get_street(x):
    x = x.replace('Երևան', '').replace('Yerevan', '').replace('Ереван', '').\
        replace(' Ave', 'Avenue').replace(' St', 'Street').replace('srteet', 'street')
    try:
        street_tuple = re.findall(
            r'улица(.*)|(.*)նրբ.|шоссе(.*)|(.*)блок|(.*)անցուղի|(.*)deadlock|(.*)ավտոմայրուղի|(.*)lane|(.*)квартал|(.*)զանգված|(.*)փակուղի|(.*)quarter|(.*)avenu|(.*)district|(.*)street|(.*)փողոց|(.*)highway|(.*)avenue|(.*)block|(.*)թաղամաս|(.*)վարչական շրջան|(.*)պողոտա|(.*)խճուղի|проспект(.*)|переулок(.*)|(.*)նրբանցք', 
            x.lower()
        )[0]
        street = ''.join(list(street_tuple)).strip()
        return street
    except: 
        if '›' in x:
            return ''.join(x.lower().split('›')).strip()
        elif 'Avan-Arinj' in x or 'Ավան-Առինջ' in x:
            return 'avan-arinj'
        else:
            return x

df['district_name'] = df['address'].apply(get_street)
df['district_name']

# convert_arm2eng_street
# arm2eng_letters_dict
# rus2eng_letters_dict
# street_district_dict


0                pushkin
1         yervand kochar
2          shahumyan 1st
3             մոլդովական
4             vardanants
              ...       
6673         tpagrichner
6674    yeznik koghbatsi
6675    zoravar andranik
6676       davtashen 4th
6677           margarian
Name: district_name, Length: 6678, dtype: object

In [210]:
def convert_rus2eng_street(street):
    street = street.lower()
    street = list(street)
    for idx in range(len(street)):
        if idx == 0 and street[idx] == 'е':
            street[idx] = 'ye'
        elif street[idx] in [' ', '-', '.', '1', 's', 't', '9']:
            pass
        else:
            if street[idx] in rus2eng_letters_dict.keys():
                street[idx] = rus2eng_letters_dict[street[idx]]
            else:
                pass
    return ''.join(street)

eng_letters = list(string.ascii_lowercase)

def detect_language(street):
    street = street.lower()
    street = list(street)
    language_list = [0, 0, 0]
    for letter in street:
        if letter in rus2eng_letters_dict.keys():
            language_list[0] += 1
        if letter in arm2eng_letters_dict.keys():
            language_list[1] += 1
        if letter in eng_letters:
            language_list[2] += 1

    if language_list.index(max(language_list)) == 0:
        return 'rus'
    elif language_list.index(max(language_list)) == 1:
        return 'arm'
    else:
        return 'eng'

In [213]:

detect_language('Կողբացու փողոց')


'arm'

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6678 entries, 0 to 6677
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   description             6678 non-null   object        
 1   address                 6678 non-null   object        
 2   seller_type             6678 non-null   object        
 3   total_description       6678 non-null   object        
 4   posted_date             6678 non-null   datetime64[ns]
 5   renewed_date            6678 non-null   object        
 6   seller_id               6678 non-null   object        
 7   estate_type             6678 non-null   object        
 8   estate_id               6678 non-null   int64         
 9   Floors in the Building  6678 non-null   int64         
 10  Floor Area              6678 non-null   int64         
 11  Renovation              6678 non-null   object        
 12  Construction Type       6678 non-null   object  

In [114]:
df['address'].head(20)

0                 Pushkin Street, Yerevan
1          Yervand Kochar street, Yerevan
2         Shahumyan 1st street 4, Yerevan
3          Մոլդովական փողոց 48/3, Yerevan
4              Vardanants Street, Yerevan
5              Բաշինջաղյան փողոց, Yerevan
6              Մամիկոնյանց փողոց, Yerevan
7          Վահրամ Փափազյան փողոց, Yerevan
8               Ադոնցի փողոց 4/3, Yerevan
9        Tsitsernakaberd Highway, Yerevan
10      Nikoghayos Adonts street, Yerevan
11       Zoravar Andranik street, Yerevan
12               Halabyan Street, Yerevan
13               Argishti Street, Yerevan
14          Nazarbekyan block 41, Yerevan
15    Aram Khachatryan street 27, Yerevan
16                Արգիշտիի փողոց, Yerevan
17        Անաստաս Միկոյանի փողոց, Yerevan
18               Tumanyan street, Yerevan
19               Թոթովենցի փողոց, Yerevan
Name: address, dtype: object

In [117]:
df[df['address'].apply(lambda x: True if 'улица' in x.lower() else False)]

,description,address,seller_type,total_description,posted_date,renewed_date,seller_id,estate_type,estate_id,Floors in the Building,...,dishwasher,yard view,view of Ararat,park view,city view,street view,concierge,playground,intercom entry,sqm_price_usd
59,"2 room apartment in a new building, 15-я улица...","15-я улица Нор Ареша, Yerevan",agency,"Վաճառվում է 45,3 մք 1-ը դարձրած 2 սենյականոց բ...",2023-12-20,2024-04-18 00:00:00,etagi,apartment,20435350,9,...,0,1,0,0,0,1,0,0,1,1266.666667
175,"4 room apartment on Mamikonyants street, 133 s...","Улица Мамиконянца, Yerevan",agency,A wonderful duplex apartment is for sale in Ar...,2024-02-23,2024-04-17 00:00:00,2385024,apartment,20686549,6,...,0,1,0,1,0,0,0,0,0,1466.165414
376,"3 room apartment, Karapet Ulnetsi Street, 90 s...","Улица Карапета Улнеци, Yerevan",agency,Վաճառվում է 3 սենյականոց բնակարան Քանաքեռ-Զեյթ...,2024-01-20,2024-04-18 00:00:00,etagi,apartment,20544541,5,...,0,1,0,0,0,1,0,0,1,1477.777778
397,"3 room apartment, Marshal Babajanyan Street, 5...","Улица Маршала Бабаджаняна, Yerevan",individual,"Վաճառվում է կապիտալ վերանորոգված, քիչ բնակեցվա...",2023-09-26,2024-04-19 00:00:00,601102,apartment,20048062,9,...,1,1,0,1,1,1,0,1,0,2272.727273
507,"3 room apartment on Yeznik Koghbatsi street, 9...","Улица Езник Кохбаци 42, Yerevan",individual,Рядом театра Оперы и балета и Северного проспе...,2023-10-12,2024-04-18 00:00:00,627157,apartment,20116661,9,...,0,1,0,0,0,1,0,0,1,2578.947368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6245,"2 room apartment on Zoravar Andranik street, 4...","Улица Андраника Зоравара, Yerevan",agency,Квартра в бангладеше.У Вагаса.,2024-04-04,2024-04-17 00:00:00,32444,apartment,20858239,9,...,0,0,0,0,0,0,0,0,0,1319.148936
6331,"3 room apartment, Улица Григора Асратяна, 75 s...","Улица Григора Асратяна 6, Yerevan",individual,"For sale, I also exchange with a car for an ad...",2024-02-10,2024-04-20 00:00:00,2408060,apartment,20633889,5,...,0,1,0,0,0,1,0,0,0,1533.333333
6358,"2 room apartment in a new building, Улица Гриб...","Улица Грибоедова 58, Yerevan",individual,"Двухкомнатная квартира, на улице Грибоедова (А...",2022-08-17,2024-04-18 00:00:00,995179,apartment,18193806,15,...,0,1,0,0,1,0,1,1,1,2142.857143
6572,"2 room apartment, Улица Захария Саркавага, 52 ...","Улица Захария Саркавага 127/1, Yerevan",individual,"The house is located near the bus stop, in the...",2024-03-10,2024-04-20 00:00:00,2679332,apartment,20751655,2,...,0,1,0,0,0,0,0,0,0,1115.384615
